In [1]:
!pip install --upgrade openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 2.12.0
    Uninstalling openai-2.12.0:
      Successfully uninstalled openai-2.12.0


In [2]:
import json
import random
import requests
from openai import OpenAI

In [3]:
class LLMClient:
    def __init__(self, provider="openai", api_key=None, base_url=None, model="gpt-4o-mini"):
        self.provider = provider
        self.api_key = api_key
        self.model = model
        self.base_url = base_url

        if provider == "openai":
            self.client = OpenAI(api_key=self.api_key)

        elif provider == "ollama":
            if not self.base_url:
                self.base_url = "http://localhost:11434"

        else:
            raise ValueError("Unknown provider. Use 'openai' or 'ollama'.")

    def generate(self, prompt):
        """Unified text generation interface (no streaming for Ollama)."""

        # ---------- OpenAI ----------
        if self.provider == "openai":
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}]
            )
            return response.choices[0].message.content.strip()

        # ---------- Ollama ----------
        elif self.provider == "ollama":
            url = f"{self.base_url}/api/generate"

            response = requests.post(
                url,
                json={
                    "model": self.model,
                    "prompt": prompt,
                    "stream": False       # <<< this fixes ALL freeze issues
                }
            )

            data = response.json()       # now safe: single JSON object
            return data["response"].strip()



In [4]:
import json
import random
import re

class RobustDataGenerator:
    # --- 1. Constants and Configuration ---

    ASPECTS = [
        "difficulty", "clarity", "workload", "lecturer_quality",
        "exam_fairness", "relevance", "interest", "support",
        "materials", "overall_experience"
    ]

    ASPECT_KEYWORDS = {
        "difficulty": ["hard", "easy", "challenging", "struggle", "breeze", "complex", "simple", "tough", "demanding", "intense", "impossible", "walk in the park", "piece of cake", "advanced", "basic", "tricky", "concept", "abstract", "theory", "sweat", "nightmare"],
        "clarity": ["explain", "understand", "confusing", "clear", "fuzzy", "articulate", "messy", "organized", "structured", "all over the place", "coherent", "follow", "lost", "crystal clear", "vague", "ambiguous", "examples", "click", "make sense"],
        "workload": ["assignments", "homework", "time", "hours", "busy", "heavy", "light", "projects", "deadlines", "overwhelmed", "drowning", "manageable", "weekend", "free time", "load", "tasks", "submission", "pressure", "burnout", "life"],
        "lecturer_quality": ["professor", "lecturer", "teach", "speaker", "explained", "engaging", "boring", "monotone", "charismatic", "passion", "enthusiastic", "cares", "knowledgeable", "expert", "delivery", "teaching style", "mentor", "vibes"],
        "exam_fairness": ["exam", "test", "grading", "grade", "score", "midterm", "final", "fair", "unfair", "curve", "questions", "representative", "surprise", "trick questions", "time limit", "prepared", "study", "pass", "fail", "cheat sheet"],
        "relevance": ["job", "industry", "real-world", "theoretical", "practical", "career", "useful", "useless", "applicable", "market", "outdated", "modern", "interview", "future", "hiring", "skills", "application", "cv"],
        "interest": ["boring", "fascinating", "sleep", "engaging", "curious", "fun", "snooze", "excited", "dry", "monotonous", "captivating", "eye-opening", "interesting", "dull", "waste of time", "love", "hate", "passion", "meh"],
        "support": ["help", "email", "office hours", "answer", "ignore", "available", "TA", "teaching assistant", "reply", "ghosted", "responsive", "feedback", "accessible", "reach out", "supportive", "on your own", "piazza"],
        "materials": ["slides", "book", "moodle", "recording", "textbook", "notes", "presentation", "resources", "video", "upload", "link", "PDF", "reading", "content", "access", "quality", "organized", "repository", "github"],
        "overall_experience": ["overall", "course", "semester", "experience", "recommend", "regret", "worth it", "avoid", "must-take", "rating", "glad", "general", "impression", "atmosphere", "vibe", "conclusion", "waste", "gem"]
    }

    SENTIMENTS = ["positive", "negative", "neutral"]

    FORBIDDEN_PHRASES = [
        "underwhelming", "overwhelmingly", "insightful", "invaluable", "facilitated",
        "comprehensive foundation", "challenging yet rewarding", "well-deserved",
        "exceeded expectations", "surpassed expectations", "notable", "commendable",
        "lackluster", "optimal", "pivotal", "foster", "enhancing", "utilize",
        "on the complexity front", "a mixed bag", "a testament to",
        "wrap my head around", "reading from a script", "monotonous voice",
        "left me struggling", "ignite a genuine sense", "solid 5/10", "mixed experience"
    ]

    STYLES = [
        "Casual (Texting style: lowercase, no punctuation, slang like 'tbh', 'idk', 'tho', fragments)",
        "Simple & Direct (Use only common words. No academic jargon. Like talking to a friend)",
        "Rant/Rave (Emotional, very subjective, lots of punctuation !!! or ...)",
        "Short (Max 10-15 words. Fragments allowed)",
        "Analytic but Simple (Explains 'why' but uses simple language)",
        "Confused Student (Unsure about things, asks rhetorical questions)",
    ]

    COURSES = [
        {"name": "Linear Algebra", "desc": "Matrices, vectors, eigenvalues."},
        {"name": "Introduction to Programming", "desc": "Python, algorithms, basic coding."},
        {"name": "Data Structures", "desc": "Trees, graphs, hash tables, complexity."},
        {"name": "Operating Systems", "desc": "Processes, threads, memory management."},
        {"name": "Machine Learning", "desc": "Neural networks, models, clustering."},
        {"name": "Computer Networks", "desc": "TCP/IP, routing, protocols."},
        {"name": "Databases", "desc": "SQL, normalization, indexing."},
        {"name": "Calculus 2", "desc": "Integrals, series, convergence tests."},
        {"name": "Probability & Statistics", "desc": "Distributions, hypothesis testing."},
        {"name": "Digital Systems", "desc": "Logic gates, boolean algebra, hardware design."},
        {"name": "Physics 1: Mechanics", "desc": "Newton's laws, energy, momentum."},
        {"name": "Algorithms 1", "desc": "Sorting, searching, dynamic programming."},
        {"name": "Web Development", "desc": "HTML, CSS, JavaScript, React."},
        {"name": "Cyber Security Basics", "desc": "Encryption, attacks, network defense."}
    ]

    LECTURERS = [
        {"name": "Dr. Cohen", "gender": "Male"},
        {"name": "Prof. Levi", "gender": "Female"},
        {"name": "Dr. Mizrahi", "gender": "Male"},
        {"name": "Dr. Kaplan", "gender": "Female"},
        {"name": "Prof. Baruch", "gender": "Male"},
        {"name": "Dr. Shapira", "gender": "Female"},
        {"name": "Dr. Adler", "gender": "Male"},
        {"name": "Prof. Klein", "gender": "Male"}
    ]

    PREFIXES = [
        "Overall, ", "To be honest, ", "Frankly, ", "In my opinion, ", "Actually, ",
        "Truth be told, ", "Looking back, ", "All in all, ", "Basically, ",
        "So, ", "Okay, so ", "Tbh, ", "Ngl, ", "Look, ", "Just saying, ",
        "Man, ", "Dude, ", "Seriously, ", "Omg, ", "Wow, ",
        "Bottom line: ", "Short version: ", "Straight up: ", "Real talk: ",
        "Just finished this course and ", "Don't take this if ", "Take this if ",
        "After finishing the semester, ", "I really thought ", "I expected ",
        "Coming into this course, ", "If I could choose again, "
    ]

    STUDENT_YEARS = [
        "Freshman (1st year)", "Sophomore (2nd year)", "Junior (3rd year)", "Senior (4th year)"
    ]
    COURSE_STATES = [
        "Currently taking", "Completed recently", "Took it a while ago", "Retaking the course"
    ]
    GRADES = [
        "A (Excellent)", "B (Good)", "C (Average)", "D (Barely passed)", "F (Failed)"
    ]

    def __init__(self, llm_client: "LLMClient", refiner_llm: "LLMClient" = None):
        self.llm = llm_client
        self.refiner_llm = refiner_llm or llm_client

    # ---------- Helpers ----------

    def get_random_course_and_lecturer(self):
        course = random.choice(self.COURSES)
        lecturer = random.choice(self.LECTURERS)
        return course["name"], course["desc"], lecturer

    # ---------- Text Post-processing (The Cleaner) ----------

    def _postprocess_review(self, text: str) -> str:
        if not text:
            return ""

        s = text.strip()

        # 1. חילוץ חכם מתוך מרכאות
        quotes_match = re.search(r'["\'](.*?)["\']', s, re.DOTALL)
        if quotes_match:
            content = quotes_match.group(1).strip()
            if len(content) > 3: # הגנה מפני תוכן קצר מדי
                s = content

        # 2. ניקוי הקדמות (Prefixes)
        lines = s.split("\n")
        clean_lines = []
        for line in lines:
            if not re.match(r'^(Here|Sure|Okay|Review|Rewrite|Style|The review)', line.strip(), re.IGNORECASE):
                clean_lines.append(line)
        s = " ".join(clean_lines).strip()

        if ":" in s[:40]:
             s = s.split(":", 1)[1].strip()

        # 3. ניקוי סיומות והערות
        s = re.sub(r'\(Note:.*?\)$', '', s, flags=re.IGNORECASE | re.DOTALL).strip()
        s = re.sub(r'\((i kept|maintained|style|sentiment|wrote|just).*?\)$', '', s, flags=re.IGNORECASE | re.DOTALL).strip()

        closers = ["Let me know", "Hope this", "Is there anything", "Feel free", "Does this meet"]
        for closer in closers:
            if closer.lower() in s.lower():
                s = re.split(closer, s, flags=re.IGNORECASE)[0].strip()

        # --- 4. ניקוי זנבות אגרסיבי (התיקון הגדול) ---
        # רשימה של מילים שאסור שיסיימו משפט
        bad_endings = [
            "and", "but", "so", "or", "because", "cuz", "tho", "although", "if", "that", # מילות קישור
            "they", "he", "she", "it", "we", "i", "you", # גופים
            "the", "a", "an", "my", "ur", "his", "her", "their", # מילות יחס
            "was", "is", "are", "were", "been", "be", # פועלי עזר
            "didn", "wasn", "weren", "isn", "aren", "can", "cant", "wont", "dont", # מילים חתוכות נפוצות
            "feelin", "hopin", "tryin", "goin", # גרונדיום חתוך
            "of", "in", "on", "at", "to", "for", "with", "about" # מילות יחס
        ]

        # לולאה שמורידה מילים אחרונות כל עוד הן ברשימה הבעייתית
        # (כי לפעמים יש רצף: "and that the...")
        words = s.split()
        while words and (words[-1].lower() in bad_endings or len(words[-1]) < 2): # מוחק גם אותיות בודדות בסוף (חוץ מ-a/i שמטופלות ברשימה)
            words.pop()

        s = " ".join(words)

        # 5. תיקון התחלות חתוכות
        s = re.sub(r"^'?m\s", "I'm ", s).strip()
        s = re.sub(r"^'?s\s", "It's ", s).strip()
        s = re.sub(r"^'?re\s", "They're ", s).strip()
        s = re.sub(r"^'?t\s", "Didn't ", s).strip()

        # השלמת n't למילים שבורות בהתחלה (כמו "t know")
        if s.startswith("t ") or s.startswith("d "):
             s = "Didn't " + s[2:]

        # 6. ניקוי סופי
        s = s.strip('"\' ,.-') # מוריד גם נקודות ופסיקים מיותרים מהקצוות
        s = " ".join(s.split())
        s = re.sub(r'\b(Dr|Prof)\.?$', '', s).strip()

        return s

    def _mess_up_text(self, text: str, style: str) -> str:
        # פונקציה שמוסיפה אופי אנושי לסגנונות לא רשמיים
        if "Casual" not in style and "Rant" not in style and "Short" not in style:
            return text

        if random.random() > 0.3:
            text = text.lower()
        if text.endswith("."):
            text = text[:-1]

        replacements = {
            "to": "2", "for": "4", "you": "u", "are": "r", "because": "cuz",
            "really": "rly", "please": "plz", "people": "ppl", "homework": "hw",
            "though": "tho", "through": "thru", "and": "&", "professor": "prof",
        }

        words = text.split()
        new_words = []
        for w in words:
            clean_w = re.sub(r"[^\w\s]", "", w).lower()
            if clean_w in replacements and random.random() > 0.4:
                new_words.append(replacements[clean_w])
            else:
                new_words.append(w)

        return " ".join(new_words)

    def _add_noise(self, text: str) -> str:
        if not text or len(text) < 15:
            return text
        if random.random() > 0.4:
            return text

        s = text
        op = random.choice(["drop_char", "remove_space"])

        if op == "drop_char" and len(s) > 5:
            # מתחילים מ-1 כדי לא למחוק את האות הראשונה בטעות
            i = random.randint(1, len(s) - 1)
            if s[i].isalpha():
                s = s[:i] + s[i + 1 :]
        elif op == "remove_space":
            match = re.search(r"[.,!?]\s+", s)
            if match:
                start, end = match.span()
                s = s[: start + 1] + s[end:]

        return s

    def _ensure_not_starting_with_i(self, text: str) -> str:
        if not text:
            return text
        s = text.lstrip()
        # מוסיפים פתיח רק אם הטקסט ארוך מספיק ומתחיל ב-I
        if len(s.split()) > 7 and re.match(r"^i\b", s, flags=re.IGNORECASE):
            prefix = random.choice(self.PREFIXES)
            s = prefix + s
        return s

    def _enforce_style_length(self, text: str, style: str) -> str:
        """Force length constraints to avoid run-on sentences in short styles."""
        words = text.split()
        if not words:
            return text

        if "Minimalist" in style:
            max_w = 6
            if len(words) > max_w:
                words = words[:max_w]
            return " ".join(words)

        if "Short" in style:
            max_w = 15
            if len(words) > max_w:
                words = words[:max_w]
            return " ".join(words)

        if len(words) > 60:
            words = words[:60]
        return " ".join(words)

    # ---------- 3. Prompts ----------

    def _build_gen_prompt(
        self,
        course_name: str,
        course_desc: str,
        lecturer_name: str,
        lecturer_gender: str,
        aspect_labels: dict,
        style: str,
        student_year: str,
        course_state: str,
        grade: str,
    ) -> str:
        aspect_instructions = []
        for aspect, sentiment in aspect_labels.items():
            keywords = ", ".join(self.ASPECT_KEYWORDS.get(aspect, []))
            instruction = f"- For '{aspect}' ({sentiment}): Use keywords like: {keywords}."

            if aspect == "clarity" and sentiment == "positive":
                instruction += " (CONSTRAINT: Do NOT use words like 'puzzle' or 'confusing')."

            if sentiment == "neutral":
                instruction += " (CONSTRAINT: Do NOT be enthusiastic. Use 'okay', 'fine', 'average', 'standard')."

            aspect_instructions.append(instruction)

        aspect_guidance_str = "\n".join(aspect_instructions)
        forbidden_str = ", ".join(self.FORBIDDEN_PHRASES)

        length_constraint = "Maximum 30 words."
        if "Minimalist" in style:
            length_constraint = "MAXIMUM 5-7 WORDS. No full sentences."
        elif "Short" in style:
            length_constraint = "Maximum 15 words. Fragments allowed."
        elif "Casual" in style or "Rant" in style:
            length_constraint = "Maximum 25 words."

        grade_instruction = ""
        if "D" in grade or "F" in grade:
            grade_instruction = "The grade is low. You can sound disappointed about the grade, BUT you must still respect the specific Aspect Sentiments."

        return f"""
Write a student review about a university course.

STYLE: {style}
LENGTH: {length_constraint}

FORBIDDEN WORDS (Do NOT use these):
{forbidden_str}

RULES:
- Sound like a REAL STUDENT. Imperfect grammar is okay.
- Do NOT start with "I" unless necessary.
- Do NOT add meta-comments.
- Minimalist style MUST be tiny.

ASPECTS TO COVER:
{json.dumps(aspect_labels, indent=2)}

DETAILS:
{aspect_guidance_str}

CONTEXT:
- Course: {course_name} ({course_desc})
- Lecturer: {lecturer_name} ({lecturer_gender})
- Student: {student_year}, {course_state}, Grade: {grade}
{grade_instruction}

Output ONLY the review text.
"""

    def _build_refiner_prompt(self, original_review: str, aspect_labels: dict, style: str) -> str:
        return f"""
You are a student. Rewrite this review to match the style: "{style}".
Original: "{original_review}"

IMPORTANT: Output ONLY the review text. NO preamble.

Constraints:
- Keep the sentiment: {json.dumps(aspect_labels)}
- If Casual: use lowercase, slang (tbh, rn), no periods.
- If Minimalist: Keep it under 6 words.
- Don't use big words.
"""

    # ---------- 4. Main API ----------

    def generate_single(self):
        # 1. בוחרים קורס ומרצה
        course_name, course_desc, lecturer_obj = self.get_random_course_and_lecturer()

        style = random.choice(self.STYLES)
        student_year = random.choice(self.STUDENT_YEARS)

        # --- שינוי 1: בחירת סטטוס קורס מאוזנת ---
        # הסרנו זמנית את האפשרות ל-Retaking כדי למנוע יצירת נכשלים אוטומטית
        # (או שאפשר להשאיר ולקוות שהכמות תהיה זניחה, אבל ליתר ביטחון ננטרל את ההשפעה שלה על הציון)
        course_state = random.choice(["Currently taking", "Completed recently", "Took it a while ago"])

        # --- שינוי 2: הגרלת ציונים מתוקנת לאיזון ---
        # אנחנו צריכים רק A, B, C. ה-F וה-D כבר מלאים.
        # המשקלים מחושבים לפי החוסרים שלך כדי להגיע ל-1200 מכל סוג.
        grade = random.choices(
            population=["A (Excellent)", "B (Good)", "C (Average)", "D (Barely passed)", "F (Failed)"],
            weights=[34, 33, 33, 0, 0], # מייצר רק ציונים גבוהים/בינוניים
            k=1
        )[0]

        # 4. דגימת אספקטים והתאמה לוגית לציון
        num_aspects = random.randint(1, 3)
        selected_aspects_list = random.sample(self.ASPECTS, num_aspects)
        aspect_labels = {}

        for a in selected_aspects_list:
            # אם נכשלים (למקרה נדיר שיברח), חוויה כללית/קושי לא יכולים להיות חיוביים
            if ("F" in grade or "D" in grade) and a in ["overall_experience", "difficulty", "exam_fairness"]:
                aspect_labels[a] = random.choice(["negative", "neutral"])
            # אם מצטיינים, חוויה כללית בד"כ לא שלילית (אלא אם רוצים ציניות)
            elif "A" in grade and a == "overall_experience":
                aspect_labels[a] = random.choice(["positive", "neutral"])
            else:
                aspect_labels[a] = random.choice(self.SENTIMENTS)

        # 5. יצירה ראשונית
        gen_prompt = self._build_gen_prompt(
            course_name,
            course_desc,
            lecturer_obj["name"],
            lecturer_obj["gender"],
            aspect_labels,
            style,
            student_year,
            course_state,
            grade,
        )
        gen_raw = self.llm.generate(gen_prompt)
        gen_text = self._postprocess_review(gen_raw)

        # 6. עידון (Refinement)
        refiner_prompt = self._build_refiner_prompt(gen_text, aspect_labels, style)
        ref_raw = self.refiner_llm.generate(refiner_prompt)
        ref_text = self._postprocess_review(ref_raw)

        # 7. גימורים (לכלוך, רעש, אכיפת אורך)
        ref_text = self._mess_up_text(ref_text, style)
        ref_text = self._ensure_not_starting_with_i(ref_text)
        ref_text = self._add_noise(ref_text)
        ref_text = self._enforce_style_length(ref_text, style)

        return {
            "course_name": course_name,
            "lecturer": lecturer_obj["name"],
            "grade": grade,
            "style": style,
            "aspects": aspect_labels,
            "review_text": ref_text,
        }

    def generate_many(self, n: int):
        return [self.generate_single() for _ in range(n)]

    @staticmethod
    def save_jsonl(dataset, filename: str):
        with open(filename, "w", encoding="utf-8") as f:
            for item in dataset:
                f.write(json.dumps(item, ensure_ascii=False) + "\n")

In [5]:
if __name__ == "__main__":

    # -------- choose model provider --------

    # ----- OpenAI -----
    # llm = LLMClient(
    #     provider="openai",
    #     api_key="YOUR_OPENAI_KEY",
    #     model="gpt-4o-mini"
    # )

    # ----- Ollama -----
    llm = LLMClient(
        provider="ollama",
        model="llama3",
        base_url="http://localhost:11434"
    )

    Robustgenerator = RobustDataGenerator(llm)

    # Generate one example
    Robustexample = Robustgenerator.generate_single()


    print(json.dumps( Robustexample, indent=4, ensure_ascii=False))


ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7e43b4d633b0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
import os
import json
import time

# הנחה: המחלקה RobustDataGenerator כבר מוגדרת למעלה בקוד שלך
# וה-llm_client שלך כבר מאותחל

def count_existing_reviews(filename):
    """פונקציית עזר לספירת שורות בקובץ קיים"""
    if not os.path.exists(filename):
        return 0
    with open(filename, 'r', encoding='utf-8') as f:
        return sum(1 for _ in f)

def generate_dataset_in_batches(generator, filename="reviews_dataset.jsonl", target_total=5000, batch_size=100):
    """
    מייצרת דאטה בנגלות ושומרת לקובץ.
    generator: המופע של המחלקה RobustDataGenerator
    filename: שם הקובץ לשמירה
    target_total: היעד הסופי של ביקורות
    batch_size: כמה לייצר בכל סיבוב (מומלץ 50-100)
    """

    # 1. בדיקה כמה כבר יש לנו (כדי לא לדרוס או לייצר סתם אם עצרנו באמצע)
    current_count = count_existing_reviews(filename)
    print(f"📂 הקובץ '{filename}' מכיל כרגע {current_count} ביקורות.")

    if current_count >= target_total:
        print("✅ היעד כבר הושג! אין צורך לייצר עוד.")
        return

    print(f"🚀 מתחיל ייצור של עוד {target_total - current_count} ביקורות...")
    print("-" * 50)

    # 2. הלולאה הראשית
    while current_count < target_total:
        # חישוב כמה נשאר כדי לא לחרוג בבאץ' האחרון
        remaining = target_total - current_count
        current_batch_size = min(batch_size, remaining)

        start_time = time.time()

        try:
            # יצירת הבאץ'
            print(f"⚙️  מייצר באץ' של {current_batch_size} ביקורות... ", end="", flush=True)
            batch_data = generator.generate_many(current_batch_size)

            # שמירה לקובץ (מצב 'a' = append)
            with open(filename, "a", encoding="utf-8") as f:
                for review in batch_data:
                    f.write(json.dumps(review, ensure_ascii=False) + "\n")

            # עדכון מונה
            current_count += len(batch_data)
            elapsed = time.time() - start_time

            print(f"✅ בוצע ({elapsed:.1f} שניות).")
            print(f"📊 סה\"כ בדאטה-סט: {current_count} / {target_total}")

        except Exception as e:
            print(f"\n❌ שגיאה בייצור הבאץ': {e}")
            print("ממתין 5 שניות ומנסה שוב...")
            time.sleep(5)
            # לא מעלים את המונה, הלולאה תנסה שוב
            continue

    print("-" * 50)
    print(f"🎉 סיימנו! הקובץ '{filename}' מכיל כעת {current_count} ביקורות.")



In [ ]:
# --- דוגמה לשימוש ---

llm = LLMClient(
    provider="ollama",
    model="llama3",
    base_url="http://localhost:11434"
)
# 1. אתחול הגנרטור (החלף את llm_client בלקוח האמיתי שלך)
my_generator = RobustDataGenerator(llm_client=llm)

# 2. הרצת הפונקציה
generate_dataset_in_batches(
    generator=my_generator,
    filename="final_student_reviews.jsonl",
    target_total=6000,
    batch_size=50  # אפשר להוריד ל-50 אם יש timeouts
 )